In [1]:
import pandas as pd
import numpy as np

from pyscf import gto, scf, ao2mo


In [100]:
element = 'K'
#url = f"https://www.molpro.net/info/basis.php?search=1&element={element}&basis=Dunning-DZP&print=1"
url = "https://www.molpro.net/info/basis.php?search=1&element=K&basis=aug-cc-pVQZ&print=1"
dfs = pd.read_html(url)
print(len(dfs))

5


In [101]:
mol = gto.Mole()
mol.atom = f'''{element}  0 0 0'''
mol.charge = 0
mol.spin = 1
mol.unit = 'AU'


In [102]:
if False:
    mol.basis = {'H': [[0,
                        [19.2406000, 0.0328280],
                        [2.8992000, 0.2312080],
                        [0.6534000, 0.8172380],],
                       [0,
                        [0.1776000, 1.0000000],],
                       [1,
                        [1.0000000, 1.0000000],]],
    }
    print(mol.basis)


In [103]:
if True:
    contraction = 0
    basis = {}
    basis[element] = []
    for l in range(0,len(dfs)):
        if len(dfs[l].columns) > 1:
            for column in range(1,len(dfs[l].columns)):
                basis[element].append([l])
                for line in np.ndarray.tolist(dfs[l].values):
                    if line[column] > 0.0:
                        basis[element][contraction].append([line[0],line[column]])
                contraction+=1
            print(contraction, column, l)
        else:
            for line in np.ndarray.tolist(dfs[l].values):
                basis[element].append([l , [line[0], 1.0]])

        

    #print(basis)
    
    mol.basis = basis 
    #mol.basis = 'sto-3g'
print(mol.basis)

8 8 0
15 7 1
20 5 2
23 3 3
25 2 4
{'K': [[0, [6741027.0, 1e-06], [2112545.0, 5e-06], [731996.6, 1.6e-05], [273831.4, 5.2e-05], [108936.2, 0.000155], [45398.78, 0.000442], [19658.22, 0.0012], [8824.135, 0.00311], [4091.672, 0.007718], [1952.908, 0.018273], [957.1958, 0.040874], [480.5229, 0.084833], [246.3229, 0.157851], [128.6348, 0.247362], [68.29012, 0.292081], [36.66851, 0.215147], [19.71727, 0.073795], [10.35031, 0.007564], [5.521912, 0.000893], [2.970577, 5e-05], [1.565147, 1.5e-05], [0.398607, 3e-06], [0.052907, 6e-06], [0.019684, 4e-06]], [0, [10.35031, 0.265027], [5.521912, 0.491469], [2.970577, 0.327769], [1.565147, 0.065421], [0.795116, 0.003404], [0.398607, 0.001256], [0.196359, 9.5e-05], [0.052907, 6.1e-05], [0.019684, 3.7e-05]], [0, [731996.6, 2e-06], [273831.4, 5e-06], [108936.2, 1.5e-05], [45398.78, 4.2e-05], [19658.22, 0.000114], [8824.135, 0.000296], [4091.672, 0.000738], [1952.908, 0.001768], [957.1958, 0.004038], [480.5229, 0.008753], [246.3229, 0.017582], [128.6348,

In [104]:
mol.build()

In [105]:
hf = scf.ROHF(mol).run()


converged SCF energy = -598.509883870499
